In [4]:
# !pip install dbnomics
# !pip install ray

In [1]:
import requests
import pandas as pd
import dbnomics
import pickle
import numpy as np
import datetime
import math 
import ray

In [3]:
final_df = pd.read_pickle('data/new_final_df.pkl')

In [4]:
''' 

weo_data() function can fetch the data for Following variables

1) General government net lending/borrowing - GGXCNL_NGDP
2) General government gross debt - GGXWDG_NGDP
3) Inflation, average consumer prices	- PCPIPCH
4) Volume of exports of goods and services - TX_RPCH
5) Gross national savings	= NGSD_NGDP

returns as list contains values as follows in sequence [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]

'''
@ray.remote
def weo_data(year):
    values_ed = []
    country_string = ''
    for country in final_df['country_code']:
        country_string+="+"+country
    country_string = country_string[1:]

    fnc = 'GGXCNL_NGDP+GGXWDG_NGDP+PCPIPCH+TX_RPCH+NGSD_NGDP'
    sr_code = country_string+'.'+fnc

    data = dbnomics.fetch_series("IMF", "WEO", series_code=sr_code, max_nb_series=1000)
    current_year_data = data.query('period=='+str(year))
    before_1_year_data = data.query('period=='+str(year-1))
    before_2_year_data = data.query('period=='+str(year-2))
    before_3_year_data = data.query('period=='+str(year-3))

    '''
    country_name as key and values as list contains [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]
    '''

    dict_values_ggxc = {}
    dict_values_ggxw = {}
    dict_values_ngsd = {}
    dict_values_pcpi = {}
    dict_values_txrp = {}


    vlist1 = []
    vlist2 = []
    vlist3 = []
    vlist4 = []
    vlist5 = []

    for country in final_df.country_code:
        cd1 = current_year_data[current_year_data['weo-country'] == country]
        cd2 = before_1_year_data[before_1_year_data['weo-country'] == country]
        cd3 = before_2_year_data[before_2_year_data['weo-country'] == country]
        cd4 = before_3_year_data[before_3_year_data['weo-country'] == country]
        if cd1.empty:
            v1 = [None,None,None,None,None]
        else:
            v1 = list(cd1['value'].values)
        if cd2.empty:
            v2 = [None,None,None,None,None]
        else:
            v2 = list(cd2['value'].values)
        if cd3.empty:
            v3 = [None,None,None,None,None]
        else:
            v3 = list(cd3['value'].values)
        if cd4.empty:
            v4 = [None,None,None,None,None]
        else:
            v4 = list(cd4['value'].values)

        for num, ver in enumerate([v1,v2,v3,v4]):
            if ver[0] == None or str(ver[0]) == 'nan':
                if num ==3:
                    dict_values_ggxc[country] = ver[0]  
                else:
                  continue
            else:
                dict_values_ggxc[country] = ver[0]
                break

        for num, ver in enumerate([v1,v2,v3,v4]):
            if ver[1] == None or str(ver[1]) == 'nan':
                if num ==3:
                    dict_values_ggxw[country] = ver[1]
                else:
                    continue
            else:
                dict_values_ggxw[country] = ver[1]
                break

        for num, ver in enumerate([v1,v2,v3,v4]):
            if ver[2] == None or str(ver[2]) == 'nan':
                if num ==3:
                    dict_values_ngsd[country] = ver[2]
                else:
                    continue
            else:
                dict_values_ngsd[country] = ver[2]
                break

        for num, ver in enumerate([v1,v2,v3,v4]):
            if ver[3] == None or str(ver[3]) == 'nan':
                if num ==3:
                    dict_values_pcpi[country] = ver[3]
                else:
                    continue
            else:
                dict_values_pcpi[country] = ver[3]
                break

        for num, ver in enumerate([v1,v2,v3,v4]):
            if ver[4] == None or str(ver[4]) == 'nan':
                if num ==3:
                    dict_values_txrp[country] = ver[4]
                else:
                    continue
            else:
                dict_values_txrp[country] = ver[4]
                break

        vlist1.append(dict_values_ggxc[country])
        vlist2.append(dict_values_ggxw[country])
        vlist3.append(dict_values_ngsd[country])
        vlist4.append(dict_values_pcpi[country])
        vlist5.append(dict_values_txrp[country])

    return [vlist1, vlist2, vlist3, vlist4, vlist5]

In [5]:
'''

7) External debt stocks, total (DOD, current US$) - DT.DOD.DECT.CD
8) Exports of goods and services (current US$) - NE.EXP.GNFS.CD
9) Total reserves (includes gold, current US$) - FI.RES.TOTL.CD
10) GDP per capita (constant 2010 US$) - NY.GDP.PCAP.KD
11) Imports of goods and services (current US$) - NE.IMP.GNFS.CD
12) GDP growth (annual %) - NY.GDP.MKTP.KD.ZG
13) GNI per capita (constant 2010 US$) - NY.GNP.PCAP.KD
14) GDP (current US$) - NY.GDP.MKTP.CD

'''
@ray.remote
def wb_data(year, indicatr):

    cr_yr_dict = {}
    before_1_yr_dict = {}
    before_2_yr_dict = {}
    before_3_yr_dict = {} 

    values_ed = []
    for i in range(6):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year)+'&format=json&page='+str(i+1)).json()
        except:
            dt0 = None
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year-1)+'&format=json&page='+str(i+1)).json()
        except:
            dt1 = None
        try:
            dt2 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year-2)+'&format=json&page='+str(i+1)).json()
        except:
            dt2 = None
        try:
            dt3 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year-3)+'&format=json&page='+str(i+1)).json()
        except:
            dt3 = None

        if dt0 != None and dt0[0]['total']!=0:
            for item in dt0[1]:
                cr_yr_dict[item['country']['id']] = item['value']
        if dt1 != None and dt1[0]['total']!=0:
            for item in dt1[1]:
                before_1_yr_dict[item['country']['id']] = item['value']
        if dt2 != None and dt2[0]['total']!=0:
            for item in dt2[1]:
                before_2_yr_dict[item['country']['id']] = item['value']
        if dt3 != None and dt3[0]['total']!=0:
            for item in dt3[1]:
                before_3_yr_dict[item['country']['id']] = item['value']

    for cntry in final_df['country_id']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        elif len(before_2_yr_dict)>0 and before_2_yr_dict[cntry] != None:
            values_ed.append(before_2_yr_dict[cntry])
        elif len(before_3_yr_dict) and before_3_yr_dict[cntry] != None:
            values_ed.append(before_3_yr_dict[cntry])
        else:
            values_ed.append(None)
  
    return values_ed

In [6]:
'''

15) Political Stability Estimate - PV.EST
16) Rule of Law Estimate - RL.EST
17) Govt Effectiveness Estimate - GE.EST

'''
@ray.remote
def pol_data(year, indicatr):
    cr_yr_dict = {}
    before_1_yr_dict = {}
    before_2_yr_dict = {}
    before_3_yr_dict = {} 
    values_ed = []
    for i in range(5):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year)+'?format=json&page='+str(i+1)).json()
        except:
            dt0 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year, i+1))
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year-1)+'?format=json&page='+str(i+1)).json()
        except:
            dt1 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-1, i+1))
        try:
            dt2 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year-2)+'?format=json&page='+str(i+1)).json()
        except:
            dt2 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-2, i+1))
        try:
            dt3 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year-3)+'?format=json&page='+str(i+1)).json()
        except:
            dt3 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-3, i+1))

        if dt0 != None:
            for item in dt0['source']['data']:
                cr_yr_dict[item['variable'][0]['id']] = item['value']
        if dt1 != None:
            for item in dt1['source']['data']:
                before_1_yr_dict[item['variable'][0]['id']] = item['value']
        if dt2 != None:
            for item in dt2['source']['data']:
                before_2_yr_dict[item['variable'][0]['id']] = item['value']
        if dt3 != None:
            for item in dt3['source']['data']:
                before_3_yr_dict[item['variable'][0]['id']] = item['value']
    
    for cntry in final_df['country_code']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        elif len(before_2_yr_dict)>0 and before_2_yr_dict[cntry] != None:
            values_ed.append(before_2_yr_dict[cntry])
        elif len(before_3_yr_dict) and before_3_yr_dict[cntry] != None:
            values_ed.append(before_3_yr_dict[cntry])
        else:
            values_ed.append(None)
    return values_ed

In [7]:
'''

15) Liquid Liabilities to GDP - GFDD.DI.05

'''
@ray.remote
def LL_GDP_data(year):
    indicatr = 'GFDD.DI.05'
    cr_yr_dict = {}
    before_1_yr_dict = {}
    before_2_yr_dict = {}
    before_3_yr_dict = {} 
    values_ed = []
    for i in range(5):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year)+'?format=json&page='+str(i+1)).json()
        except:
            dt0 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year, i+1))
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year-1)+'?format=json&page='+str(i+1)).json()
        except:
            dt1 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-1, i+1))
        try:
            dt2 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year-2)+'?format=json&page='+str(i+1)).json()
        except:
            dt2 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-2, i+1))
        try:
            dt3 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year-3)+'?format=json&page='+str(i+1)).json()
        except:
            dt3 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-3, i+1))

        if dt0 != None:
            for item in dt0['source']['data']:
                cr_yr_dict[item['variable'][0]['id']] = item['value']
        if dt1 != None:
            for item in dt1['source']['data']:
                before_1_yr_dict[item['variable'][0]['id']] = item['value']
        if dt2 != None:
            for item in dt2['source']['data']:
                before_2_yr_dict[item['variable'][0]['id']] = item['value']
        if dt3 != None:
            for item in dt3['source']['data']:
                before_3_yr_dict[item['variable'][0]['id']] = item['value']
    
    for cntry in final_df['country_code']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        elif len(before_2_yr_dict)>0 and before_2_yr_dict[cntry] != None:
            values_ed.append(before_2_yr_dict[cntry])
        elif len(before_3_yr_dict) and before_3_yr_dict[cntry] != None:
            values_ed.append(before_3_yr_dict[cntry])
        else:
            values_ed.append(None)
    return values_ed

In [8]:
clms = ['fiscal_balance_gdp', 'government_grossdebt_gdp', 'inflation_data', 'exports_growth', 'savings_gdp', 'externaldebt',
               'exports', 'reserves', 'real_gdppc', 'imports', 'real_gdpgrowth', 'real_gnipc', 'nominal_gdp', 'political_stability', 'rule_of_law',
               'govt_effectiveness', 'm3_gdp'] #, 'reer']

def fetch_data(year):
    try:
        ray.init()
        c_year = datetime.datetime.today().year 
        year = year - 1
        final_dfd = pd.read_pickle('new_final_df.pkl')
        if c_year >= year:
    #     [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth] = weo_data(year)

    #     externaldebt = wb_data(year, 'DT.DOD.DECT.CD')
    #     exports = wb_data(year, 'NE.EXP.GNFS.CD')
    #     political_stability = pol_data(year, 'PV.EST')

    #     reserves = wb_data(year, 'FI.RES.TOTL.CD')
    #     real_gdppc = wb_data(year, 'NY.GDP.PCAP.KD')
    #     rule_of_law = pol_data(year, 'RL.EST')

    #     imports = wb_data(year, 'NE.IMP.GNFS.CD')
    #     real_gdpgrowth = wb_data(year, 'NY.GDP.MKTP.KD.ZG')
    #     govt_effectiveness = pol_data(year, 'GE.EST')

    #     real_gnipc = wb_data(year, 'NY.GNP.PCAP.KD')
    #     nominal_gdp = wb_data(year, 'NY.GDP.MKTP.CD')

    #     m3_gdp = LL_GDP_data(year)

            ret_id1 = weo_data.remote(year)
            ret_id2 = wb_data.remote(year, 'DT.DOD.DECT.CD')
            ret_id3 = wb_data.remote(year, 'NE.EXP.GNFS.CD')
            ret_id4 = pol_data.remote(year, 'PV.EST')
            ret_id5 = wb_data.remote(year, 'FI.RES.TOTL.CD')
            ret_id6 = wb_data.remote(year, 'NY.GDP.PCAP.KD')
            ret_id7 = pol_data.remote(year, 'RL.EST')
            ret_id8 = wb_data.remote(year, 'NE.IMP.GNFS.CD')
            ret_id9 = wb_data.remote(year, 'NY.GDP.MKTP.KD.ZG')
            ret_id10 = pol_data.remote(year, 'GE.EST')

            ret_id11 = wb_data.remote(year, 'NY.GNP.PCAP.KD')
            ret_id12 = wb_data.remote(year, 'NY.GDP.MKTP.CD')

            ret_id13 = LL_GDP_data.remote(year)


            [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth], externaldebt, exports, political_stability, reserves, real_gdppc, rule_of_law, imports, real_gdpgrowth, govt_effectiveness, real_gnipc, nominal_gdp, m3_gdp = ray.get([ret_id1, ret_id2, ret_id3, ret_id4, ret_id5, ret_id6, ret_id7, ret_id8, ret_id9, ret_id10, ret_id11, ret_id12, ret_id13])
        #     [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth] = ret1 
            # reer = EREER_IX_data(year)

            lt_118 = []
            ind_lst = [fiscal_balance_gdp, government_grossdebt_gdp, inflation_data, exports_growth, savings_gdp, externaldebt,
                       exports, reserves, real_gdppc, imports, real_gdpgrowth, real_gnipc, nominal_gdp, political_stability, rule_of_law,
                       govt_effectiveness, m3_gdp] #, reer]

            for nm, indctr in enumerate(ind_lst):
                if len(indctr) < 118:
                    lt_118.append(clms[nm])

            for col,vals in zip(clms, ind_lst):
                if vals not in lt_118:
                    final_dfd[col] = vals

            final_dfd['externaldebt/exports'] = (final_dfd['externaldebt']/final_dfd['exports'])*100
            final_dfd['reserves/import'] = (final_dfd['reserves']/final_dfd['imports'])*100
            final_dfd['reserves/gdp'] = (final_dfd['reserves']/final_dfd['nominal_gdp'])*100
            final_dfd['log_real_gnipc'] = final_dfd['real_gnipc'].apply(lambda x: math.log(x))
            final_dfd['predicted_specgrade'] = (final_dfd['log_real_gnipc']*(-5.3914)+final_dfd['savings_gdp']*(-0.1321)+final_dfd['rule_of_law']*(-4.9736)+final_dfd['government_grossdebt_gdp']*(0.1253)+46.7664)/math.sqrt(1+(7.463)**2)
            final_dfd['Speculative_Grade_Prob'] = final_dfd['predicted_specgrade'].apply(lambda x: math.exp(x)/(1+math.exp(x)))
            final_dfd['y_hat'] = final_dfd['externaldebt/exports']*( -0.0081)+final_dfd['real_gdppc']*(0.0005)+final_dfd['reserves/import']*(0.02867)+final_dfd['fiscal_balance_gdp']*(0.06526)+final_dfd['political_stability']*(1.09237)+final_dfd['default_hist']*(-0.50728)+final_dfd['govt_effectiveness']*(1.3579)+final_dfd['real_gdpgrowth']*(0.09908)+final_dfd['inflation_data']*(-0.03967)
            final_dfd['(Threshold_Yhat)/sigma']= final_dfd['y_hat'].apply(lambda x: ((-5.2355)-x)/math.sqrt(27.252))
            final_dfd['Debt_Distress_prob'] = final_dfd['(Threshold_Yhat)/sigma'].apply(lambda x: math.exp(x)/(1+math.exp(x)))
            ray.shutdown()
            return final_dfd 
        else:
            print("The Year You have Passed is in Future, Please use the Current Year or Past Year for Made this Working")
    except:
        print("Some Error Occured")
        ray.shutdown()
    finally:
        ray.shutdown()

In [26]:
%%time
dtt = fetch_data(2016)

2020-09-10 14:33:19,647	INFO resource_spec.py:231 -- Starting Ray with 2.29 GiB memory available for workers and up to 1.15 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-10 14:33:20,058	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


CPU times: user 2.21 s, sys: 1.04 s, total: 3.25 s
Wall time: 1min 3s


In [23]:
dtt

,country_id,country_code,country_name,default_hist,fiscal_balance_gdp,government_grossdebt_gdp,inflation_data,exports_growth,savings_gdp,externaldebt,...,m3_gdp,externaldebt/exports,reserves/import,reserves/gdp,log_real_gnipc,predicted_specgrade,Speculative_Grade_Prob,y_hat,(Threshold_Yhat)/sigma,Debt_Distress_prob
0,AF,AFG,Afghanistan,0,0.118,7.823,4.384,-17.135,26.772,2.596712e+09,...,32.7072,228.581116,99.378141,37.608041,NaN,NaN,NaN,-3.242117,-0.381849,0.405681
1,AO,AGO,Angola,0,-4.520,75.663,30.694,-0.662,24.487,5.716739e+10,...,35.9042,201.006529,92.725239,23.409108,8.122965,1.942439,0.874620,-0.724539,-0.864112,0.296481
2,AL,ALB,Albania,0,-1.762,73.324,1.282,10.387,16.791,8.741215e+09,...,72.9772,254.317530,57.183238,26.208199,8.466958,1.291247,0.784358,2.477470,-1.477483,0.185808
3,AE,ARE,United Arab Emirates,0,-2.810,20.217,1.617,6.063,20.002,NaN,...,90.5634,NaN,31.574490,23.915982,10.627884,-1.974134,0.121946,NaN,NaN,NaN
4,AR,ARG,Argentina,1,-6.655,53.060,10.619,6.807,16.340,1.887576e+11,...,24.3082,270.261836,50.786571,6.890109,9.211931,0.471299,0.615691,3.261393,-1.627650,0.164153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,VE,VEN,Venezuela,1,-10.834,5.054,254.949,-11.729,4.015,1.525229e+11,...,42.6290,189.412384,6.702046,2.104241,9.541206,0.873217,0.705415,-8.908544,0.703602,0.668986
114,VN,VNM,Vietnam,0,-3.081,59.662,2.668,10.723,29.520,8.566260e+10,...,139.4410,44.572376,19.540700,17.794219,7.423902,1.320427,0.789253,1.665849,-1.322010,0.210484
115,YE,YEM,Yemen,0,-9.224,79.591,21.300,-69.037,-1.718,7.062886e+09,...,32.7087,NaN,NaN,19.029227,NaN,NaN,NaN,NaN,NaN,NaN
116,ZA,ZAF,South Africa,0,-4.067,51.465,6.341,0.352,16.291,1.460410e+11,...,43.1176,161.141896,52.965627,15.920016,8.891117,0.338562,0.583841,3.735707,-1.718509,0.152063


In [15]:
dtest = pd.read_json('https://raw.githubusercontent.com/jayeshmanani/TensorFlow/master/hs.json')

,code3,name,value,code
0,ABW,Aruba,582.34,AW
1,AFG,Afghanistan,53.08,AF
2,AGO,Angola,23.11,AO
3,ALB,Albania,104.97,AL
4,AND,Andorra,164.43,AD
...,...,...,...,...
208,WSM,Samoa,68.95,WS
209,YEM,"Yemen, Rep.",52.25,YE
210,ZAF,South Africa,46.18,ZA
211,ZMB,Zambia,22.32,ZM


In [69]:
dt2 = pd.read_pickle('../data_website/firstUI/data/2019.pkl')

In [70]:
dt2

,country_id,country_code,country_name,default_hist,fiscal_balance_gdp,government_grossdebt_gdp,inflation_data,exports_growth,savings_gdp,externaldebt,...,m3_gdp,externaldebt/exports,reserves/import,reserves/gdp,log_real_gnipc,predicted_specgrade,Speculative_Grade_Prob,y_hat,(Threshold_Yhat)/sigma,Debt_Distress_prob
0,AF,AFG,Afghanistan,0,-0.983,7.547,2.302,-17.605,19.248,2.605358e+09,...,33.8223,175.909266,117.481568,44.487191,NaN,NaN,NaN,-2.618245,-0.501357,0.377222
1,AO,AGO,Angola,0,0.695,94.993,17.079,0.656,20.994,5.456337e+10,...,29.8246,131.830920,63.099727,17.260825,8.000524,2.387099,0.915838,-0.202453,-0.964122,0.276054
2,AL,ALB,Albania,0,-1.957,66.677,1.411,1.736,17.408,1.012192e+10,...,71.2840,210.246651,54.480278,24.634389,8.466958,1.211673,0.770595,3.072580,-1.591481,0.169176
3,AE,ARE,United Arab Emirates,0,-0.814,20.135,-1.931,3.067,37.712,NaN,...,84.8594,NaN,37.572912,25.729870,10.636530,-2.264257,0.094127,NaN,NaN,NaN
4,AR,ARG,Argentina,1,-3.869,93.301,53.548,9.892,17.328,2.805157e+11,...,25.4298,360.802090,66.577098,10.055437,9.143250,1.071327,0.744849,0.809576,-1.157984,0.239034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,VE,VEN,Venezuela,1,-10.007,182.448,19906.020,-38.665,17.039,1.548981e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,VN,VNM,Vietnam,0,-3.302,54.346,2.797,11.669,28.524,1.080963e+11,...,145.3190,38.644426,28.867207,29.907782,7.557322,1.205853,0.769564,2.141930,-1.413207,0.195729
115,YE,YEM,Yemen,0,-3.823,56.263,10.000,43.381,3.917,7.036567e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,ZA,ZAF,South Africa,0,-6.254,59.873,4.130,2.673,14.517,1.793064e+11,...,42.9300,170.904998,53.375816,15.666174,8.872545,0.667374,0.660915,3.422051,-1.658425,0.159974


In [41]:
dt2.Debt_Distress_prob = dt2.Debt_Distress_prob * 100

In [61]:
dt3 = dt2[['country_code','Debt_Distress_prob']].rename(columns={'country_code':'code3','Debt_Distress_prob':'value'})

In [67]:
ft = dtest.drop(columns=['value']).merge(dt3, on='code3', how='left')[['code3','name','value','code']]

In [103]:
ft.iloc[0]

code3      ABW
name     Aruba
value      NaN
code        AW
Name: 0, dtype: object

In [105]:
lst = []
for i in range(len(ft)):
    lst.append({"code3": ft.iloc[i]['code3'],
		"name": ft.iloc[i]['name'],
		"value": ft.iloc[i]['value'],
		"code": ft.iloc[i]['code']})

In [106]:
lst

[{'code3': 'ABW', 'name': 'Aruba', 'value': nan, 'code': 'AW'},
 {'code3': 'AFG',
  'name': 'Afghanistan',
  'value': 37.72218892113776,
  'code': 'AF'},
 {'code3': 'AGO', 'name': 'Angola', 'value': 27.605373947824013, 'code': 'AO'},
 {'code3': 'ALB', 'name': 'Albania', 'value': 16.91756348030786, 'code': 'AL'},
 {'code3': 'AND', 'name': 'Andorra', 'value': nan, 'code': 'AD'},
 {'code3': 'ARE', 'name': 'United Arab Emirates', 'value': nan, 'code': 'AE'},
 {'code3': 'ARG',
  'name': 'Argentina',
  'value': 23.903379852603074,
  'code': 'AR'},
 {'code3': 'ARM',
  'name': 'Armenia',
  'value': 20.181543986912832,
  'code': 'AM'},
 {'code3': 'ASM', 'name': 'American Samoa', 'value': nan, 'code': 'AS'},
 {'code3': 'ATG', 'name': 'Antigua and Barbuda', 'value': nan, 'code': 'AG'},
 {'code3': 'AUS', 'name': 'Australia', 'value': nan, 'code': 'AU'},
 {'code3': 'AUT', 'name': 'Austria', 'value': nan, 'code': 'AT'},
 {'code3': 'AZE',
  'name': 'Azerbaijan',
  'value': 16.33271381943457,
  'code'

In [107]:
import numpy as np

In [110]:
np.linspace(0,100,8)

array([  0.        ,  14.28571429,  28.57142857,  42.85714286,
        57.14285714,  71.42857143,  85.71428571, 100.        ])